In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## Criando o Cliente

Neste exemplo, usaremos [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) para acessar o LLM.

O `model` está definido como `gpt-4o-mini`. Experimente alterar o modelo para outro disponível no marketplace do GitHub Models para ver os diferentes resultados.

Como um teste rápido, vamos executar um prompt simples - `Qual é a capital da França`.


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## Definindo as Funções

Neste exemplo, daremos ao agente acesso a uma ferramenta que é uma função com uma lista de destinos de férias disponíveis e suas respectivas disponibilidades.

Você pode imaginar que este seria um cenário onde um agente de viagens poderia ter acesso, por exemplo, a um banco de dados de viagens.

Conforme você explora este exemplo, sinta-se à vontade para tentar definir novas funções e ferramentas que o agente possa chamar.


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## Definindo a Ferramenta de Função  
Para que o agente utilize o `vacation_destinations` como um `FunctionTool`, precisamos defini-lo como tal.  

Também forneceremos uma descrição da ferramenta, o que ajuda o agente a identificar para que ela é usada em relação à tarefa solicitada pelo usuário.


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## Definindo o agente

Agora podemos criar o agente no código abaixo. Definimos a `system_message` para fornecer ao agente instruções sobre como ajudar os usuários a encontrar destinos de férias.

Também configuramos o parâmetro `reflect_on_tool_use` como verdadeiro. Isso permite que o LLM utilize a resposta da chamada da ferramenta e envie a resposta usando linguagem natural.

Você pode configurar o parâmetro como falso para ver a diferença.


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automáticas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
